### Goals

1. Add more Tools / Agents to enhance capabilities
   - Add Another Tool for booking
2. Add an Agent that translates all responses to a different language and shows on the right side, try with different Frontier Model
3. Add and Agent that can listen for Audio and convert it to Text

# Add More Tools

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [6]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [7]:
### Price Tool

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
### Handle Price Tool

def handle_price_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [9]:
### Booking Tool

def book_flight():
    print("Ticket has been booked")
    return {'status': 200}

book_function = {
    "name": "book_flight",
    "description": "Book a flight for a destination. Call this whenever the user requests to book the flight, for example when a customer asks 'Can you help me book tihs flight?'",
    "parameters": {}
}

In [10]:
def handle_book_flight_call(message):
    tool_call = message.tool_calls[0]
    status = book_flight()
    response = {
        "role": "tool",
        "content": json.dumps({"status": "Ticket has been booked"}),
        "tool_call_id": tool_call.id
    }
    return response, status

In [11]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": book_function}]

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        if response.choices[0].message.tool_calls[0].function.name=="get_ticket_price":
            message = response.choices[0].message
            response, city = handle_price_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        elif response.choices[0].message.tool_calls[0].function.name=="book_flight":
            message = response.choices[0].message
            response, status = handle_book_flight_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [13]:
chat("Great, can you help me book the flight?", 
    [
        {"role": "user", "content": "Hi, how's it going?"}, 
        {"role": "assistant", "content": "Hello! I'm here to help you with your travel needs. How can I assist you today?"},
        {"role": "user", "content": "How about Paris?"}, 
        {"role": "assistant", "content": "Could you please specify what information you would like about Paris?"},
        {"role": "user", "content": "What are things to see there?"}, 
        {"role": "assistant", "content": "In Paris, you can explore the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and enjoy a stroll along the Seine River."},
        {"role": "user", "content": "Can you tell me the ticket price for a flight to Paris?"}, 
        {"role": "assistant", "content": "The ticket price for a flight to Paris is approximately $899."}        
    ]
)

Ticket has been booked


'Your flight to Paris has been successfully booked!'

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Ticket has been booked


# Add Tool Functionality with Cluade

In [12]:
import os
import json
from dotenv import load_dotenv
import anthropic
import gradio as gr

In [14]:
load_dotenv(override=True)

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
MODEL = "claude-3-7-sonnet-latest"
client = anthropic.Anthropic(api_key=anthropic_api_key)

In [60]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "The user will send all messages in English. Respond only in Spanish."

In [33]:
## Antropic version of chat function
history_store = []

def chat(message, history):
    global history_store

    # Convert Gradio's history format to Anthropic format
    messages = []
    for turn in history_store:
        messages.append({"role": "user", "content": turn["user"]})
        messages.append({"role": "assistant", "content": turn["assistant"]})

    messages.append({"role": "user", "content": message})
    
    response = client.messages.create(system=system_message, model=MODEL, max_tokens=1024, messages=messages)

    assistant_response = ""
    
    for block in response.content:
        if block.type == "text":
            assistant_response += block.text    

    history_store.append({"user": message, "assistant": assistant_response})
    
    return assistant_response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [ ]:
## With Tools
tools = [
    {
        "name": "get_ticket_price",
        "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to"
                }
            },
            "required": ["destination_city"]
        }
    },
    {
        "name": "book_flight",
        "description": "Book a flight for a destination. Call this whenever the user requests to book the flight, for example when a customer asks 'Can you help me book this flight?'",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to"
                }
            },
            "required": ["destination_city"]
        }
    }    
]

def execute_tool(name, input_data):
    if name == "get_ticket_price":
        ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
        city = input_data.get("destination_city", "Unknown City")
        print(f"Tool get_ticket_price called for {city}")
        city = city.lower()
        price = ticket_prices.get(city, "Unknown")
        # return {"destination_city": city,"price": price}
        return f"The ticket for a flight to {city} is {price}"
    elif name == "book_flight":
        print("Ticket has been booked")
        # return {"ticket_status": "Successfully booked"}
        return "Ticket has been booked"

history_store = []

def chat(message, history):
    global history_store

    # Convert Gradio's history format to Anthropic format
    messages = []
    for turn in history_store:
        messages.append({"role": "user", "content": turn["user"]})
        messages.append({"role": "assistant", "content": turn["assistant"]})

    messages.append({"role": "user", "content": message})
    
    response = client.messages.create(system=system_message, model=MODEL, max_tokens=1024, tools=tools, messages=messages)

    assistant_response = ""
    tool_call = None
    
    for block in response.content:
        if block.type == "text":
            assistant_response += block.text   
        elif block.type == "tool_use":
            tool_call = block        

   # If tool is requested
    if tool_call:
        tool_result_data = execute_tool(tool_call.name, tool_call.input)

        # Send tool output back to Claude
        tool_followup = client.messages.create(
            model=MODEL,
            system=system_message,
            max_tokens=1024,
            messages=[
                *messages,
                {
                    "role": "assistant",
                    "content": [
                        {
                            "type": "tool_use",
                            "id": tool_call.id,
                            "name": tool_call.name,
                            "input": tool_call.input
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_call.id,
                            "content": tool_result_data
                        }
                    ]
                }
            ]
        )
        
        assistant_response = "".join(
            block.text for block in tool_followup.content if block.type == "text"
        )        

    history_store.append({"user": message, "assistant": assistant_response})
    
    return assistant_response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7897
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Ticket has been booked


# Add translation functionality

Here, what I want to do is very simply copy the same functionality from OpenAI's chat, but just add an additional part which also translates and just adds the translation to a new line. Example below:

```
import gradio as gr
from transformers import pipeline

# Initialize translation pipeline (English to French)
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

# Main chatbot response function
def respond(message, history):
    # Simulate LLM response (you can replace this with OpenAI or other LLM API call)
    llm_response = f"You said: {message}"

    # Translate the response to French
    translation = translator(llm_response, max_length=100)[0]['translation_text']

    # Combine original and translated response
    full_response = f"{llm_response}\n\n_Translated:_ {translation}"

    return full_response

# Gradio ChatInterface
chat = gr.ChatInterface(fn=respond, 
                        title="Bilingual Chat",
                        description="This chatbot replies and also translates its responses to French.")

chat.launch()
```


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

In [11]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [33]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "You are also trying to teach Spanish. The user will se sending their english and spanish translation."
system_message += "Always reply back with an English and Spanish version of the response like this:"
system_message += "Response: {English response}"
system_message += "Translated: {Spanish response}"

In [34]:
### Price Tool

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [35]:
### Handle Price Tool

def handle_price_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [36]:
### Booking Tool

def book_flight():
    print("Ticket has been booked")
    return {'status': 200}

book_function = {
    "name": "book_flight",
    "description": "Book a flight for a destination. Call this whenever the user requests to book the flight, for example when a customer asks 'Can you help me book tihs flight?'",
    "parameters": {}
}

In [37]:
def handle_book_flight_call(message):
    tool_call = message.tool_calls[0]
    status = book_flight()
    response = {
        "role": "tool",
        "content": json.dumps({"status": "Ticket has been booked"}),
        "tool_call_id": tool_call.id
    }
    return response, status

In [38]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": book_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        if response.choices[0].message.tool_calls[0].function.name=="get_ticket_price":
            message = response.choices[0].message
            response, city = handle_price_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        elif response.choices[0].message.tool_calls[0].function.name=="book_flight":
            message = response.choices[0].message 
            response, status = handle_book_flight_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Ticket has been booked


# Agent to listen for Audio and convert to Text

In [11]:
import sounddevice as sd
from scipy.io.wavfile import write


sd.default.device = 1 ## Sets the recording device to my microphone

# Settings
samplerate = 44100  # 44.1 kHz
duration = 5        # seconds
filename = "recording.wav"

print("Recording...")
audio_data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
sd.wait()  # Wait until recording is finished
print("Recording complete. Saving file...")

write(filename, samplerate, audio_data)  # Save as WAV file
print(f"File saved as: {filename}")


Recording...
Recording complete. Saving file...
File saved as: recording.wav


In [12]:
audio_file = open("recording.wav", "rb")
transcript = openai.audio.transcriptions.create(
  model="gpt-4o-transcribe",
  file=audio_file
)

In [13]:
transcript.text

"Let's go Yankees."

# Test Summarize

In [16]:
from pydub import AudioSegment
import os

def chunk_m4a_audio(input_file, output_dir, chunk_length_sec=30):
    """
    Splits an .m4a audio file into multiple chunks.

    Parameters:
    - input_file: str, path to the input .m4a file
    - output_dir: str, directory to save output chunks
    - chunk_length_sec: int, chunk size in seconds (default: 30s)
    
    Returns:
    - List of output chunk file paths
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    print(f"🔄 Loading audio file: {input_file}")
    audio = AudioSegment.from_file(input_file, format="m4a")
    total_length = len(audio)
    chunk_length_ms = chunk_length_sec * 1000

    print(f"📏 Total audio length: {total_length / 1000:.2f} seconds")
    chunk_paths = []

    for i in range(0, total_length, chunk_length_ms):
        chunk = audio[i:i + chunk_length_ms]
        chunk_filename = f"chunk_{i // chunk_length_ms:04d}.m4a"
        chunk_path = os.path.join(output_dir, chunk_filename)
        chunk.export(chunk_path, format="mp4")
        print(f"✅ Exported: {chunk_path} ({len(chunk) / 1000:.2f}s)")
        chunk_paths.append(chunk_path)

    print("🎉 Chunking complete.")
    return chunk_paths


In [17]:
chunk_paths = chunk_m4a_audio(
    input_file=r"D:\python-development\udemy_llms\llm_engineering\week2\audio\inputfiles\AFCNorthBreakdown.m4a", 
    output_dir=r"C:\test",
    chunk_length_sec=1200
    )

🔄 Loading audio file: D:\python-development\udemy_llms\llm_engineering\week2\audio\inputfiles\AFCNorthBreakdown.m4a
📏 Total audio length: 3382.36 seconds
✅ Exported: C:\test\chunk_0000.m4a (1200.00s)
✅ Exported: C:\test\chunk_0001.m4a (1200.00s)
✅ Exported: C:\test\chunk_0002.m4a (982.36s)
🎉 Chunking complete.


In [12]:
audio_file = open(r".\audio\outputfiles\chunk_0002.m4a", "rb")
transcript = openai.audio.transcriptions.create(
  model="gpt-4o-transcribe",
  file=audio_file
)

In [13]:
transcript.text

"...for 93 targets and he caught 86% of them for 8 yards of reception. I mean, he was just so valuable to Joe Burrow in that check down game and I don't see that going away. Like from week 4 through 17, because he didn't play week 18. From weeks 4 through 17, he has one game under 10 points. And you have multiple smash games in there. I love Chase Brown this year. Just circling back to test my theory here and get your opinions. All right. T. Higgins' current ADP relative to some of these other players. You like Garrett Wilson more or you like T. Higgins more? Higgins. Oh, man. Garrett Wilson was slightly better in consistency last year percentage-wise, but is a couple picks later. You like Higgins, Mike? I think we'd all like him over McLaurin. Yeah. Who's going later? Marvin Harrison Jr. is going later. I'll still take Higgins. Evans, Devante Adams, a couple of the older guys, DJ Morashi Rice. Yeah. It doesn't seem... It's high risk, but very high reward. Yeah, I don't see a bunch of 

In [14]:
complete_output += " " + transcript.text

In [15]:
complete_output

"Welcome in. Thursday, July 3rd, we are the fantasy footballers Mike Wright, Jason Moore, Andy Holloway. The crew is back together. We are 10 Sundays from football. Wait, wait, wait. Like real football? Yeah, from like the season. The first week, I think it was Scott Hansen that tweeted that out. It's the first weekend of the red zone. I mean, if you're like, hey, 10 weeks till the Hall of Fame game. No, get out of here. Get out of here. I'll be excited, but I don't need to count out for that. That's fair. I mean, we are still very close to meaningful football, and the Hall of Fame game is this month. We're in the month of the Hall of Fame game. It was last week. It was a heck of a game. I haven't gone that long. Is it this month? Yeah. It's in July? July 31st this year. What is happening with the world? Like my daughter. Yeah, they're going back to school in July, and I've got a real problem with this. My daughter starts school in July. What? What are we doing? Why even stop school? J

In [17]:
import re

def chunk_text(text, num_chunks=4):
    # Split text into sentences using punctuation
    sentences = re.split(r'(?<=[.!?]) +', text)
    total_sentences = len(sentences)
    chunk_size = total_sentences // num_chunks
    
    chunks = []
    for i in range(num_chunks):
        start = i * chunk_size
        # Ensure the last chunk gets any leftover sentences
        end = (i + 1) * chunk_size if i < num_chunks - 1 else total_sentences
        chunk = ' '.join(sentences[start:end]).strip()
        chunks.append(chunk)
    
    return chunks


In [18]:
chunks = chunk_text(complete_output)

In [21]:
len(chunks)

4

In [20]:
system_prompt = "You are a helpful assistant that summarizes transcripts."
system_prompt += "You will get a chunk of a conversation between three fantasy football analysts."
system_prompt += "You're job is to summarize each chunk giving the most important key points highlighting any mentioned football players."

In [36]:
messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Summarize the following transcript: {chunks[3]}"}
]


In [37]:
response = openai.chat.completions.create(model=MODEL, messages=messages)

In [38]:
response.choices[0].message.content

'The analysts discuss the struggles of a particular football team, indicating that they are ranked 31st, which they agree is "bad." They note the team\'s difficult upcoming schedule and mention their ranking of 32nd in points per game from the previous year, despite having the highest pass percentage in the league. \n\nKey topics include:\n- The team has lost significant players like Nick Chubb and Jameis Winston, creating uncertainty in their offense.\n- Joe Flacco and Kenny Pickett are mentioned as potential quarterbacks, with analysts debating their impact on offensive players like Jerry Judy and David Njoku.\n- Deontay Johnson\'s comments about not wanting to play in the cold due to concerns about his performance are discussed humorously.\n- They highlight Quinchon Judkins as a promising player, considering him a strong replacement for Nick Chubb.\n\nOverall, the conversation revolves around the impact of quarterback changes on player performance and the need for careful considerat

In [39]:
full_response += response.choices[0].message.content

In [41]:
from IPython.display import Markdown, display

display(Markdown((full_response)))

On July 3rd, fantasy football analysts Mike Wright, Jason Moore, and Andy Holloway discussed the upcoming NFL season, noting that they were just 10 Sundays away from real football. They highlighted the Hall of Fame game on July 31st, which is the first game of the season. 

The conversation turned to player evaluations, particularly focusing on Derek Henry, who seems to be undervalued in drafts despite having strong scoring potential. They mentioned that Henry may be less favored in drafts compared to other players like Christian McCaffrey due to concerns about reception counts and his role in the passing game.

The analysts previewed the tough matchups for teams early in the season, such as the opening game for a team against Buffalo and other challenging opponents like Detroit and Kansas City in the following weeks.

They also discussed the prospects of wide receiver Zay Flowers, who they deemed a fine choice for a fifth-round pick, though not overwhelmingly exciting. They noted his potential to be a flexible option if he could improve on his target percentage. 

Lastly, they touched on the Pittsburgh Steelers, mentioning their recent struggles despite a strong previous season where they exceeded expectations. The team hasn't won a playoff game since 2016 and ended the last season with a disappointing final stretch.In this conversation among fantasy football analysts, the key points revolve around team performance, player evaluations, and strategies for the upcoming season. Here are the main highlights:

1. **Projected Win Total**: The focus is on a team with a projected win total of 8.5, where the analysts discuss the difficulty of their schedule, especially in their final four games, which included tough opponents like the Philadelphia Eagles, Baltimore Ravens, Kansas City Chiefs, and Cincinnati Bengals.

2. **Quarterback Comparisons**: There is a debate on the relative merits of Aaron Rodgers and Russell Wilson, with a consensus that Rodgers is a better quarterback, suggesting a lack of faith in Wilson's recent performances.

3. **Offensive Struggles**: The team ranked poorly in key offensive metrics last year, including 20th in points per game and 28th in passing percentage. This sets a low bar for improvement in the upcoming season.

4. **Coaching Strategies**: The discussion touches on the tactical differences between Aaron Rodgers and head coach Arthur Smith, particularly regarding the balance between the running game and passing game. Analysts convey skepticism about how this dynamic will play out.

5. **Pittsburgh Steelers Reputation**: They discuss Mike Tomlin's consistent performance over his 18-season tenure and some significant offseason moves, including the addition of players like Jalen Ramsey and DK Metcalf, suggesting a potential shift in team strategy.

6. **Rookie Prospects**: Caleb Johnson is highlighted as a rookie running back expected to bring more explosiveness to the backfield, especially in replacing Najee Harris. Analysts predict his ADP (average draft position) will rise as the season approaches.

7. **Fantasy Relevance**: The conversation shifts to fantasy implications, with conflicting views on DK Metcalf's potential under Rodgers' leadership, and uncertainty surrounding other offensive players like Jonu Smith and Pat Freiermuth.

8. **Backfield Dynamics**: There is debate on the roles of running backs Warren and Caleb Johnson, with analysts suggesting that Warren may start the season as the primary back.

Overall, the analysts provide a mix of optimism and skepticism about the team's potential for the upcoming season, with specific attention to new player dynamics and their impacts on both team performance and fantasy relevance.In this discussion among fantasy football analysts, several key players and topics emerge:

1. **Jalen Warren and Caleb Johnson**: Warren is highlighted as a strong pass protector whom quarterback Aaron Rodgers will appreciate. However, Caleb Johnson's explosive play ability is deemed essential for the offense as the season progresses.

2. **Team Schedules**: There's speculation that the team will start the season 3-0, with winnable matchups against the Jets, Seahawks, and Patriots. The analysts discuss the implications of a potential loss to the Jets in Week 1.

3. **Tee Higgins vs. Garrett Wilson**: The conversation shifts to Tee Higgins’ average draft position (ADP), with analysts favoring Higgins over Wilson and other receivers. Higgins is noted for his consistency and strong finish from the previous season.

4. **Teams and Quarterbacks**: The Bengals' schedule is analyzed, with an emphasis on challenges in the early season, including matchups against the Browns and Jaguars. There’s concern over the potential slow start for quarterback Joe Burrow.

5. **Cincinnati Bengals' Receiving Core**: The analysts debate the depth behind the starting receivers, with Tyler Boyd mentioned as the third option. Mike Gesicki is recommended as a potential free agency target if injuries occur in the Bengals' receiving group.

6. **Cleveland Browns Context**: The analysts touch on the struggles of the Browns, mentioning Deshaun Watson's injury and performance issues, while discussing their weak projected win total and tough strength of schedule.

In summary, the analysts express cautious optimism about Jalen Warren and Caleb Johnson, highlight Tee Higgins' draft value, discuss the Bengals' and Browns' upcoming challenges, and consider the fantasy implications of these discussions.The analysts discuss the struggles of a particular football team, indicating that they are ranked 31st, which they agree is "bad." They note the team's difficult upcoming schedule and mention their ranking of 32nd in points per game from the previous year, despite having the highest pass percentage in the league. 

Key topics include:
- The team has lost significant players like Nick Chubb and Jameis Winston, creating uncertainty in their offense.
- Joe Flacco and Kenny Pickett are mentioned as potential quarterbacks, with analysts debating their impact on offensive players like Jerry Judy and David Njoku.
- Deontay Johnson's comments about not wanting to play in the cold due to concerns about his performance are discussed humorously.
- They highlight Quinchon Judkins as a promising player, considering him a strong replacement for Nick Chubb.

Overall, the conversation revolves around the impact of quarterback changes on player performance and the need for careful consideration in drafting players from this team.